# Домашняя работа 3. Бустинг

Максимальная оценка 10 баллов

In [1]:
!pip install numpy
import numpy as np
import pandas as pd

from sklearn.datasets import load_boston # sorry
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from scipy.optimize import minimize_scalar

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### Задание 1. Градиентный бустинг своими руками  (4 балла)

Вам нужно реализовать упрощенный вариант градиентного бутсинга для задачи регресси. 


**Напоминание, как это работает:**

Обозначим текущую композицию на $N-1$ шаге за $a_{N - 1}(x_i)$. Базовый алгоритм $b_N(x_i)$ обучается на ответах $-\frac{\partial L(y_i, z)}{\partial z}\Bigl|_{z = a_{N - 1}(x_i)}$, где $L(y_i, z)$ — значение функции потерь на объекте при правильном ответе $y_i$ и предсказании $z$. Композиция на следующем шаге получается так:

$$
a_N(x_i) = a_{N-1}(x_i) + \nu\gamma_Nb_N(x_i)
$$

Здесь $\nu \in [0, 1]$ — темп обучения (гиперпараметр), $\gamma_N$ — оптимальный вес, настраиваемый на каждом шаге алгоритма в ходе решения оптимизационной задачи:

$$
\gamma_N = \mathrm{arg}\min_\gamma \frac{1}{\ell}\sum\limits_{i=1}^{\ell}L\left(y_i, a_{N - 1}(x_i) + \gamma b_N(x_i)\right)
$$


Заметьте, что в формуле выше нет $\nu$. Этот гиперпараметр используется для сокращения длины шага, оптимального при составлении композиции $a_N$. Идея отклонения от оптимума должна быть вам уже знакома как способ борьбы с переобучением, когда мы специально форсим модель работать чуть хуже, чем могла бы, на текущем шаге, чтобы сохранить обобщающую способность и не подогнаться под тренировочную выборку (или под шум).

С потерей в 0.5 балла можете принять $\gamma_N = 1$ для каждого $N$. На полный балл необходимо реализовать нахождение оптимального $\gamma_N$ на каждом шаге.

В качестве функции потерь $L$ возьмите MSE.

В качестве базовой модели можете использовать `DecisionTreeRegressor` из `sklearn`.
Для решения оптимизационной задачки можно воспользоваться алгоритмами из любых библиотек, например, `scipy.optimize`, или найти оптимум перебором по сетке из некоторого разумного диапазона.

Можно дописывать свои функции, если необходимо.

In [52]:
class GradientBoosting:
    def __init__(
        self, 
        base_model_class: object = DecisionTreeRegressor,
        base_model_params: dict = {'max_depth': None}, 
        n_estimators: int = 200,
        learning_rate: float = 0.07
    ):
        """
        
        Args:
          base_model_class: Class of the base learner.

          base_model_params: Hyperparameters of the base learner.
          
          n_estimators: Number of boosting stages.
          
          learning_rate: Value used to shrink contribution of each base learner to the model. 
          
        """
        
        self.base_model_class = base_model_class
        self.base_model_params = base_model_params
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        
        # list for optimal gammas at each iteration
        self.gammas = []
        
        # list for base models
        self.models = []
        
        
    def find_optimal_gamma(self, 
                           y: np.array, 
                           old_predictions: np.array,
                           new_predictions: np.array) -> float:
        """You may add arguments if it's necessary for your optimization algorithm.
        
        Args:
          y: Target variable.

          old_predictions: Prediction of the additive model at the previous stage.
          
          new_predictions: Prediction of the base learner at the current stage. 
          
        Returns:
          Optimal value for gamma.
          
        """
        #записываем функцию, которую минимизируем по гамме (x)
        def L(x):
            return np.sum((y - (old_predictions + x*new_predictions))**2)
        #используем minimize_scalar
        return minimize_scalar(L,bounds=(-1, 1), method='bounded').x
        
        pass
    
    
    def _fit_base_model(self, X: np.ndarray, y: np.array):
        """Train one base learner. 
        
        Args:
          X: Feature matrix
          
          y: Target variable.
          
          
        Returns:
          Fitted base learner.
          
        """
        #создаем базовое дерево 
        dt = DecisionTreeRegressor(**self.base_model_params, random_state=19052019)
        #обучем его
        dt.fit(X,y)
        #сохраняем дерево в массиве моделей
        self.models.append(dt)
        return dt
        pass
    
        
    def fit(self, X: np.ndarray, y: np.array):
        """Train boosting ("sum" of base learners). 
        
        Args:
          X: Feature matrix
          
          y: Target variable.
          
          
        Returns:
          Fitted boosting.
          
        """
        #начальная модель (инициализируем нулем)
        a = np.zeros_like(y)
        # первую базовое дерево обучаем на таргетах
        grad = y
        #запускаем цикл, который обновляет нашу модель
        for i in range(self.n_estimators):
          #сохраняем старое значение модели 
          old = a
          #обучаем новую модель и записываем предсказание ошибки
          pred = self._fit_base_model(X,grad).predict(X)
          #рассчитываем необходимую gamma 
          gamma = self.find_optimal_gamma(y,old,pred)
          #обновляем модель
          a += gamma*pred
          #записываем gamma в массив 
          self.gammas.append(gamma)
          #обновляем градиент для нового дерева
          grad = self.learning_rate*(y - a)
        return self.models
        pass
       
        
    def predict(self, X: np.ndarray):
        """Make prediction of fitted boosting. 
        
        Args:
          X: Feature matrix


        Returns:
          Prediction of fitted boosting.
          
        """
        pred = 0
        #считаем в цикле итоговое предсказание как сумму произведений предсказаний моделей на соответствующие gamma
        for i in range(self.n_estimators):
            pred += self.models[i].predict(X)*self.gammas[i]
        return pred

        pass

Проверьте вашу реализацию на бостонском датасете. Подберите оптимальные гиперпараметры, чтобы победить RandomForestRegressor (не меняйте параметры сида).

In [21]:
boston = load_boston()
X = boston.data
y = boston.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=13)

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np


        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California housing dataset (i.e.

In [4]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(max_features=4, n_estimators=640, random_state=19052019)

rf.fit(X_train, y_train)
mean_squared_error(y_test, rf.predict(X_test))

9.63198271791959

In [34]:
min = np.inf
md = 0
lr = 0
ne = 0
for i in range(3, 11):
    for j in range(1, 85, 5):
          for k in range(70, 300, 20):
              gb = GradientBoosting(base_model_params={'max_depth':i}, learning_rate=j/100, n_estimators=k) 
              gb.fit(X_train, y_train) 
              curr = mean_squared_error(y_test, gb.predict(X_test))
              if curr < min:
                  min = curr
                  md = i
                  lr = j/100
                  ne = k
print(f"MSE={min}, lr = {lr}, md = {md}, ne = {ne}")

MSE=12.46497187637409, lr = 0.51, md = 6, ne = 90


### Задание 2. Сравнение подходов (3 балла)

Скачайте данные о выдаче кредитов. Это данные с kaggle, целевая переменная `y` показывает, вернуло ли кредит физическое лицо.

In [41]:
!wget  -O 'bank_data.csv' -q 'https://www.dropbox.com/s/uy27mctxo0gbuof/bank_data.csv?dl=0'

In [42]:
df = pd.read_csv('bank_data.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9280 entries, 0 to 9279
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             9280 non-null   int64  
 1   job             9280 non-null   object 
 2   marital         9280 non-null   object 
 3   education       9280 non-null   object 
 4   default         9280 non-null   object 
 5   housing         9280 non-null   object 
 6   loan            9280 non-null   object 
 7   contact         9280 non-null   object 
 8   month           9280 non-null   object 
 9   day_of_week     9280 non-null   object 
 10  duration        9280 non-null   int64  
 11  campaign        9280 non-null   int64  
 12  pdays           9280 non-null   int64  
 13  previous        9280 non-null   int64  
 14  poutcome        9280 non-null   object 
 15  emp.var.rate    9280 non-null   float64
 16  cons.price.idx  9280 non-null   float64
 17  cons.conf.idx   9280 non-null   f

Решите задачу предсказания возвращения кредита методами, перечисленными ниже:

- Случайный лес
- Бэггинг на деревьях (поставьте для базовых деревьев min_samples_leaf=1)
- Бэггинг, у которого базовой моделью является бустинг с большим числом деревьев (> 100)
- Бэггинг на логистических регрессиях

Используйте логистическую регрессию, случайный лес, `GradientBoostingClassifier` и `BaggingClassifier` из `sklearn`.

1) Какая из моделей имеет лучшее качество? С чем это связано?

2) Какая из моделей сильнее всего переобучается?


In [43]:
from sklearn.preprocessing import OneHotEncoder

In [44]:
#разделяем данные и удаляем плохую фичу duration
X_train,X_test,y_train,y_test = train_test_split(df[df.drop(columns = ['y','duration'],axis = 1).columns],df['y'],test_size=0.25, random_state=777,stratify=df['y'])

In [45]:
#обработка данных 
types = X_train.dtypes
col = types[types[:,]=='object'].index

# X_train
enc_1 = OneHotEncoder(handle_unknown='ignore')
enc_1.fit(df[df.drop(columns = ['y'],axis = 1).columns][col])

names = np.concatenate(np.array(enc_1.categories_))
data_1 = pd.DataFrame(enc_1.transform(X_train[col]).toarray())
data_1.columns = names

#склейка таблиц:
data_1.loc[:, "N"] = np.arange(0,len(data_1),1)
dat = X_train.drop(columns = col,axis = 1)
dat.loc[:, "N"] = np.arange(0,len(data_1),1)
data_0 = dat.merge(data_1,left_on='N', right_on='N',how="outer")
data_0.drop(columns = ['N'],axis = 1,inplace=True)

# X_test
types_2 = X_test.dtypes
col_1 = types_2[types_2[:,]=='object'].index

names_2 = np.concatenate(np.array(enc_1.categories_))
data_2 = pd.DataFrame(enc_1.transform(X_test[col_1]).toarray())
data_2.columns = names_2

#склейка таблиц:
data_2.loc[:, "N"] = np.arange(0,len(data_2),1)
dat_2 = X_test.drop(columns = col_1,axis = 1)
dat_2.loc[:, "N"] = np.arange(0,len(data_2),1)
data_0_test = dat_2.merge(data_2,left_on='N', right_on='N',how="outer")
data_0_test.drop(columns = ['N'],axis = 1,inplace=True)


<ipython-input-45-dabba69ff442>:9: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  names = np.concatenate(np.array(enc_1.categories_))
<ipython-input-45-dabba69ff442>:24: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  names_2 = np.concatenate(np.array(enc_1.categories_))


In [46]:
from sklearn.metrics import roc_curve, auc
from sklearn. metrics import precision_recall_curve
from sklearn.metrics import accuracy_score

Случайный лес

In [47]:
from sklearn.ensemble import RandomForestClassifier
rf2 = RandomForestClassifier(max_features=1, n_estimators=10, random_state=19052019)

rf2.fit(data_0, y_train)

#accuracy на тесте
print(accuracy_score(y_test,rf2.predict(data_0_test)))
#accuracy на трене (показатель переобучаемости)
print(accuracy_score(y_train,rf2.predict(data_0)))






0.7060344827586207
0.9770114942528736


Беггинг на деревьях

In [48]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bc1 = BaggingClassifier().fit(data_0, y_train)

#accuracy на тесте
print(accuracy_score(y_test,bc1.predict(data_0_test)))
#accuracy на трене (показатель переобучаемости)
print(accuracy_score(y_train,bc1.predict(data_0)))



0.7060344827586207
0.9767241379310345


Бэггинг, у которого базовой моделью является бустинг с большим числом деревьев (> 100)

In [49]:
from sklearn.ensemble import GradientBoostingClassifier
bc2 = BaggingClassifier(GradientBoostingClassifier(n_estimators=100)).fit(data_0, y_train)

#accuracy на тесте
print(accuracy_score(y_test,bc2.predict(data_0_test)))
#accuracy на трене (показатель переобучаемости)
print(accuracy_score(y_train,bc2.predict(data_0)))

0.743103448275862
0.7564655172413793


Бэггинг на логистических регрессиях

In [51]:
from sklearn.linear_model import LogisticRegression

bc3 = BaggingClassifier(LogisticRegression()).fit(data_0, y_train)

#accuracy на тесте
print(f"accuracy = {accuracy_score(y_test,bc3.predict(data_0_test))}")
#accuracy на трене (показатель переобучаемости)
print(f"accuracy = {accuracy_score(y_train,bc3.predict(data_0))}")
#здесь плохо выводит print (он внутри кода)

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _che

accuracy = 0.7306034482758621
accuracy = 0.7234195402298851


/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


    1)Лучшее качество у "Бэггинг, у которого базовой моделью является бустинг с большим числом деревьев (> 100)"

    2)Сильнее всего переобучается "Случайный лес"



### Задание 3. Современные бустинги (3 балла)

Сравните на этих данных любую из трёх популярных имплементаций градиентного бустинга (xgboost, lightgbm, catboost). Подберите основные гиперпараметры (число деревьев, длина шага, глубина дерева/число листьев). Получилось ли круче, чем с моделями выше?

In [36]:
#скачиваем библиотеку
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [37]:
from catboost import CatBoostClassifier
from sklearn.metrics import classification_report
from pandas.api.types import is_numeric_dtype
import catboost as cb

X = df[df.drop(columns = ['y','duration'],axis = 1).columns]
y = df['y']

#работа с категориальными признаками (поиск)
def get_categorical_indicies(X):
    cats = []
    for col in X.columns:
        if is_numeric_dtype(X[col]):
            pass
        else:
            cats.append(col)
    cat_indicies = []
    for col in cats:
        cat_indicies.append(X.columns.get_loc(col))
    return cat_indicies
categorical_indicies = get_categorical_indicies(X)

#конвертируем категориальные признаки
def convert_cats(X):
    cats = []
    for col in X.columns:
        if is_numeric_dtype(X[col]):
            pass
        else:
            cats.append(col)
    cat_indicies = []
    for col in cats:
        X[col] = X[col].astype('category')
convert_cats(X)

#делим данные на обучающую и тестовую
X_train,X_test,y_train,y_test = train_test_split(X,
                                                 y,
                                                 test_size=0.2,
                                                 random_state=101,
                                                 stratify=y)

#переводим в формат, понятный CatBoost
train_dataset = cb.Pool(X_train,y_train,
                        cat_features=categorical_indicies)                                                     
test_dataset = cb.Pool(X_test,y_test,          
                       cat_features=categorical_indicies)
                      

model = cb.CatBoostClassifier(loss_function='Logloss', 
                              eval_metric='Accuracy')

#находим оптимальные гиперпараметры
grid = {'learning_rate': np.linspace(0, 1, 10).tolist(),
        'depth': [4, 6, 10],
        'l2_leaf_reg': [1, 3, 5,],
        'iterations': [50, 100, 150]}
       
p = model.grid_search(grid,train_dataset,plot=True)
pred = model.predict(X_test)
print(classification_report(y_test, pred))


<ipython-input-37-cbfca8a11cea>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = X[col].astype('category')


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Выходные данные были обрезаны до нескольких последних строк (5000).
35:	learn: 0.8679912	test: 0.7367003	best: 0.7542088 (2)	total: 920ms	remaining: 1.64s
36:	learn: 0.8701802	test: 0.7346801	best: 0.7542088 (2)	total: 943ms	remaining: 1.6s
37:	learn: 0.8708537	test: 0.7346801	best: 0.7542088 (2)	total: 966ms	remaining: 1.57s
38:	learn: 0.8732110	test: 0.7340067	best: 0.7542088 (2)	total: 989ms	remaining: 1.55s
39:	learn: 0.8772521	test: 0.7299663	best: 0.7542088 (2)	total: 1.01s	remaining: 1.52s
40:	learn: 0.8780940	test: 0.7313131	best: 0.7542088 (2)	total: 1.03s	remaining: 1.49s
41:	learn: 0.8833137	test: 0.7326599	best: 0.7542088 (2)	total: 1.06s	remaining: 1.46s
42:	learn: 0.8841556	test: 0.7353535	best: 0.7542088 (2)	total: 1.09s	remaining: 1.45s
43:	learn: 0.8844923	test: 0.7346801	best: 0.7542088 (2)	total: 1.12s	remaining: 1.42s
44:	learn: 0.8860077	test: 0.7299663	best: 0.7542088 (2)	total: 1.14s	remaining: 1.39s
45:	learn: 0.8988045	test: 0.7286195	best: 0.7542088 (2)	total:

In [40]:

print(f"accuracy = {accuracy_score(y_test, pred)}")

accuracy = 0.7419181034482759
